In [1]:
import torch
import time
import torch.nn as nn 
import torch.nn.functional as F
import d2lzh as d2l 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def nin_block(in_channels,out_channels,kernel_size,stride,padding):
    blk = nn.Sequential(
        nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding),
        nn.ReLU(),
        nn.Conv2d(out_channels,out_channels,kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels,out_channels,kernel_size=1),
        nn.ReLU()
        )
    return blk 

class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可以将池化窗口设置为输入的宽和高实现
    def __init__(self):
        super(GlobalAvgPool2d,self).__init__()
    def forward(self,x):
        return F.avg_pool2d(x,kernel_size=x.size()[2:])

net = nn.Sequential(
    nin_block(1,96,kernel_size=11,stride=4,padding=0),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nin_block(96,256,kernel_size=5,stride=1,padding=2),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nin_block(256,384,kernel_size=3,stride=1,padding=1),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nn.Dropout(0.5),
    nin_block(384,10,kernel_size=3,stride=1,padding=1),
    GlobalAvgPool2d(),
    d2l.FlattenLayer()
)

In [4]:
x = torch.rand(1,1,224,224)
for name,blk in net.named_children():
    x = blk(x)
    print(name,'output shape:',x.shape)

0 output shape: torch.Size([1, 96, 54, 54])
1 output shape: torch.Size([1, 96, 26, 26])
2 output shape: torch.Size([1, 256, 26, 26])
3 output shape: torch.Size([1, 256, 12, 12])
4 output shape: torch.Size([1, 384, 12, 12])
5 output shape: torch.Size([1, 384, 5, 5])
6 output shape: torch.Size([1, 384, 5, 5])
7 output shape: torch.Size([1, 10, 5, 5])
8 output shape: torch.Size([1, 10, 1, 1])
9 output shape: torch.Size([1, 10])


In [2]:
batch_size = 64
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size,resize=224)

lr,num_epochs = 0.002,5
optimizer = torch.optim.Adam(net.parameters(),lr=lr)
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cuda
epoch 1, loss 1.0409, train acc 0.619,test acc 0.781,time 108.8
epoch 2, loss 0.2574, train acc 0.813,test acc 0.822,time 105.2
epoch 3, loss 0.1459, train acc 0.840,test acc 0.844,time 105.6
epoch 4, loss 0.1010, train acc 0.850,test acc 0.851,time 105.8
epoch 5, loss 0.0760, train acc 0.858,test acc 0.853,time 105.8
